Import required dependencies

In [1]:
import os
import json
import requests
import pandas as pd
import pickle           # Save and load data
import time


# Setting Pandas options.
pd.set_option("display.max_rows", 50) # How to display all rows from data frame using pandas. Setting value to None to show all rows.
pd.set_option("display.max_columns", None)
pd.set_option("display.max_info_columns", 100)
pd.set_option("display.max_info_rows", 1000000)
pd.set_option("display.precision", 2)
#pd.set_option("styler.format.precision", 2)

load config files

In [2]:
# Load API Key and download directory from config file
CONFIG_FILE = "../config/api-call.json"

def load_config(config_path):
    """Load configuration from a JSON file."""
    with open(config_path, "r") as file:
        return json.load(file)

Config parameters for API calls

In [3]:
# Load config values
config = load_config(CONFIG_FILE)
# print(config)
API_ENDPOINT= config["ned"]["ned_api_endpoint"]
API_KEY = config["ned"]["demo-ned-api-key"]
DOWNLOAD_DIR = config["ned"]["ned_download_dir"]

# https://api.ned.nl/v1/utilizations?point=0&type=2&granularity=3&granularitytimezone=1&classification=2&activity=1&validfrom[strictly_before]=2020-11-17&validfrom[after]=2020-11-16


# Ensure the download directory exists
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# API details (Replace with actual API endpoint)
API_URL = API_ENDPOINT
#HEADERS = {"Authorization": f"X-AUTH-TOKEN {API_KEY}"}

# Headers with X-AUTH-TOKEN
api_headers = {
    "X-AUTH-TOKEN": API_KEY,  # Replace with your actual token
    "Content-Type": "application/json"  # Optional, if sending JSON
}

Make API calls in loop

In [ ]:

# Type	    What is the type of energy carrier?	0 All, 1 Wind, 2 Solar, 3 Biogas, 4 HeatPump, 8 Cofiring, 9 Geothermal, 10 Other, 11 Waste, 12 BioOil, 13 Biomass
# 14 Wood, 17 WindOffshore, 18 FossilGasPower, 19 FossilHardCoal, 20 Nuclear, 21 WastePower, 22 WindOffshoreB, 23 NaturalGas, 24 Biomethane, 25 BiomassPower
# 26 OtherPower, 27 ElectricityMix, 28 GasMix, 31 GasDistribution, 35 CHP Total, 50 SolarThermal, 51 WindOffshoreC, 53 IndustrialConsumersGasCombination
# 54 IndustrialConsumersPowerGasCombination, 55 LocalDistributionCompaniesCombination, 56 AllConsumingGas


# Define an array of n values (Custom values instead of a fixed range)
n_values = [0, 1, 2, 3, 4, 8, 9, 10, 11, 12, 13, 14, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 35, 50, 51, 53, 54, 55, 56]  # Type values
# n_values = [54, 55, 56]  # Type values
# n_values = [23, 31, 54, 55, 56]  # Type values

# n_values = [1, 2]  # Type values

# n_values = [56]  # Type values

# Initialize storage for full dataset
full_data = []
page_no = 1

# Loop through the n values
for gen_type in n_values:
    #params = {"n": n}  # Pass n as a parameter
    api_params={
        "point": "0",
        "type": gen_type,
        "granularity": "5",
        "granularitytimezone": "1",
        "classification": "2",
        "activity": "1",
        "validfrom[strictly_before]": "2025-04-14",
        "validfrom[after]": "2022-01-01",
        "page": page_no
        }

    # Initialize storage for full dataset
    full_data = []

    # First API request to determine the last page
    response = requests.get(API_URL, params=api_params, headers=api_headers)

    if response.status_code == 200:
        data = response.json()
        last_page_url = data.get("hydra:view", {}).get("hydra:last", None)
        
        if last_page_url:
            last_page_num = int(last_page_url.split("page=")[-1])  # Extract last page number
            print("last_page_num : ", last_page_num)
        else:
            print("Error: Could not determine last page.")
            exit()
    else:
        print(f"Error {response.status_code}: {response.text}")
        exit()

    # Iterate through pages efficiently using the 'page' parameter
    for page in range(1, last_page_num + 1):
        api_params["page"] = page  # Update page number
        response = requests.get(API_URL, params=api_params, headers=api_headers)
        
        if response.status_code == 200:
            json_dict = json.loads(response.content.decode("utf-8"))
        
            # Extract the list of records from 'hydra:member'
            if "hydra:member" in json_dict:
                full_data.extend(json_dict["hydra:member"])
        
            # print(f"Page {page}/{last_page_num} processed.")
        else:
            print(f"Error on page {page}: {response.status_code}")
            break  # Stop if an error occurs

        # Convert all collected data into a DataFrame
        df = pd.DataFrame(full_data)

        file_path = os.path.join(DOWNLOAD_DIR, f"power-gen-type-2025-{gen_type}.csv")  # Change extension as needed
        with open(file_path, "wb") as file:
            df.to_csv(file_path, index=False)
            # file.write(response.content)
            # print(f"File downloaded successfully: {file_path}")   
        
        # print(f"Data collection complete! Saved {len(df)} records to ned_api_data.csv & ned_api_data.pkl")
    print(f"Data collection complete! Saved {len(df)} records to ", f"power-gen-type-2025-{gen_type}.csv")
    # time.sleep(3)


        

Data collection complete! Saved 2495 records to  power-gen-type-2025-25.csv
last_page_num :  18
Data collection complete! Saved 2495 records to  power-gen-type-2025-26.csv
last_page_num :  18
Data collection complete! Saved 2495 records to  power-gen-type-2025-27.csv
last_page_num :  18
Data collection complete! Saved 2495 records to  power-gen-type-2025-28.csv
Error: Could not determine last page.
Data collection complete! Saved 0 records to  power-gen-type-2025-31.csv
last_page_num :  18
Data collection complete! Saved 2495 records to  power-gen-type-2025-35.csv
last_page_num :  18
Data collection complete! Saved 2495 records to  power-gen-type-2025-50.csv
last_page_num :  18
Data collection complete! Saved 2495 records to  power-gen-type-2025-51.csv
Error: Could not determine last page.
Data collection complete! Saved 0 records to  power-gen-type-2025-53.csv
Error: Could not determine last page.
Data collection complete! Saved 0 records to  power-gen-type-2025-54.csv
Error: Could no

: 

Functionl to make API calls